#  SQL Queries 02

In [ ]:
import pandas as pd

In [ ]:
import numpy as np

In [ ]:
%load_ext sql

In [ ]:
%sql sqlite://

## Create toy data set

In [ ]:
from faker import Faker

In [ ]:
fake = Faker()

In [ ]:
names = list(set([fake.name() for i in range(100)]))

In [ ]:
np.random.seed(123)

n1 = 30
bios821 = pd.DataFrame(dict(
    name=np.random.choice(names, n1, replace=False), 
    grade=np.random.randint(50, 101, n1)))

n2 = 30
bios823 = pd.DataFrame(dict(
    name=np.random.choice(names, n2, replace=False), 
    grade=np.random.randint(50, 101, n2)))

In [ ]:
%%sql

DROP TABLE IF EXISTS bios821;
DROP TABLE IF EXISTs bios823

In [ ]:
%sql -p bios821

In [ ]:
%sql -p bios823

In [ ]:
%sql SELECT * FROM bios821 LIMIT 3

In [ ]:
%sql SELECT * FROM bios823 LIMIT 3

## Subqueries

There are 3 ways to use a sub-query:

- with `in`
- with `exists`
- with a comparison operator

**1**. What students take both bios821 and bios823?

#### Using set operations.

In [ ]:
%%sql

SELECT name FROM bios821
INTERSECT
SELECT name FROM bios823

#### Using EQUIJOIN.

In [ ]:
%%sql

SELECT DISTINCT bios821.name 
FROM bios821, bios823
WHERE bios821.name = bios823.name

#### Using sub-query.

In [ ]:
%%sql

SELECT DISTINCT name FROM bios821
WHERE name IN (
    SELECT name FROM bios823
)

#### Using correlated sub-query.

In [ ]:
%%sql

SELECT DISTINCT name FROM bios821
WHERE EXISTS (
    SELECT * FROM bios823 
    WHERE bios821.name = bios823.name
)

**2**. Which students taking bios823 have grades higher than the average grade of bios821? 

- Do this for all students
- Do this just for the subet of patinets taking both classes

In [ ]:
%%sql

SELECT DISTINCT name, grade FROM bios823
WHERE grade > (
    SELECT AVG(grade) FROM bios821 
)

#### Using a view.

In [ ]:
%%sql

CREATE VIEW view_common AS 
    SELECT DISTINCT
        bios821.name, 
        bios821.grade as grade_821, 
        bios823.grade as grade_823 
    FROM 
        bios821, bios823
    WHERE 
        bios821.name = bios823.name
    AND bios821.name IN (
        SELECT name 
        FROM bios823 
    )     

In [ ]:
%%sql

SELECT name, grade_823
FROM view_common
WHERE grade_823 > (
    SELECT AVG(grade_821) 
    FROM view_common
)

#### Using `pandas`.

In [ ]:
df = bios821.merge(bios823, on='name', suffixes=['_821', '_823'])

In [ ]:
df

In [ ]:
df[df.grade_823 > df.grade_821.mean()]

## Common table expressions (CTE)

CTEs are temporary tables created for a specific query.

### Basic syntax for CTE

In [ ]:
%%sql

WITH
    t1(name) AS (SELECT 'Bob'),
    t2(age) AS (SELECT 23)
SELECT * from t1, t2

#### Using CTEs to solve previous problem

In [ ]:
%%sql

WITH common AS (
    SELECT DISTINCT 
        bios821.name, 
        bios821.grade as grade_821, 
        bios823.grade as grade_823 
    FROM 
        bios821,
        bios823
    WHERE bios821.name IN (
        SELECT name FROM bios823
    ) AND
    bios821.name = bios823.name
)
SELECT name, grade_823
FROM common
WHERE grade_823 > (
    SELECT AVG(grade_821) 
    FROM common
)

### Recursive CTEs

Image source: https://cdn.sqlservertutorial.net/wp-content/uploads/SQL-Server-Recursive-CTE-execution-flow.png
![img](https://cdn.sqlservertutorial.net/wp-content/uploads/SQL-Server-Recursive-CTE-execution-flow.png)

#### As a generator

In [ ]:
%%sql

WITH RECURSIVE count(n) AS (
    SELECT 1
        UNION ALL
    SELECT n+1 FROM count WHERe n < 5
)
SELECT * FROM count

#### Generating dates

In [ ]:
%%sql

WITH RECURSIVE make_dates(d) AS (
    SELECT '2019-01-15'
        UNION ALL
    SELECT DATE(d, '+1 MONTHS') FROM make_dates WHERe d < '2019-12'
)
SELECT d AS "Homework due" FROM make_dates

#### CTEs to do programming in SQL(!)

In [ ]:
%%sql

WITH RECURSIVE fact(n, f) AS (
    SELECT 1, 1
        UNION ALL
    SELECT n+1, (n+1) * f 
    FROM fact 
    WHERe n < 5
)
SELECT * FROM fact
LIMIT 5

In [ ]:
%%sql

WITH RECURSIVE fib(n, f1, f2) AS (
    SELECT 1, 0, 1
        UNION ALL
    SELECT n+1, f2, f1+f2
    FROM fib 
    WHERe n < 10
)
SELECT * FROM fib
LIMIT 10

#### A common use of CTEs is to work with naturally recursive structures (trees or graphs)

Suppose we have a cell subset taxonomy with parent-child relations. We want to find all `descendants` of a particular cell type. This is hard to do without recursive CTEs in SQL. 

In [ ]:
%%sql sqlite:///

DROP TABLE IF EXISTS cell;

CREATE TABLE cell(
    cell_id integer PRIMARY KEY,
    name VARCHAR(30),
    parent_id integer
);

INSERT INTO cell (
   cell_id,
   name,
   parent_id
)
VALUES
   (1, 'WBC', NULL),
   (2, 'Lymphocyte', 1),
   (3, 'T Cell', 2),
   (4, 'B Cell', 2),
   (5, 'NK Cell', 2),
   (6, 'T helper cell', 3),
   (7, 'T cytotoxic cell', 3),
   (8, 'T regulatory cell', 3),
   (9, 'Naive B cell', 4),
   (10, 'Memory B cell', 4),
   (11, 'Plasma cell', 4),
   (12, 'Granulocyte', 1),
   (13, 'Basophil', 12),
   (14, 'Eosinophil', 12),
   (15, 'Neutrophil', 12)

In [ ]:
%%sql

SELECT * FROM cell LIMIT 10

In [ ]:
%%sql

    SELECT cell_id, name, parent_id
    FROM cell
    WHERE name='T Cell'

In [ ]:
target = 'Lymphocyte'

In [ ]:
%%sql

WITH RECURSIVE lineage AS (
    SELECT 
        cell_id, 
        name, 
        parent_id
    FROM 
        cell
    WHERE 
        name=:target
UNION ALL
    SELECT
        c.cell_id, 
        c.name, 
        c.parent_id
    FROM 
        cell c
    INNER JOIN 
        lineage l 
    ON 
        l.cell_id = c.parent_id
)
SELECT * FROM lineage 
LIMIT 10

## Window Functions

- [Official Refs](https://www.sqlite.org/windowfunctions.html)
![img](https://www.sqlitetutorial.net/wp-content/uploads/2018/11/SQLite-Window-Functions-1.png)

In [ ]:
np.random.seed(23)
n = 10
df = pd.DataFrame(
    dict(person=np.random.choice(['A', 'B', 'C', 'D'], n,),
                time=np.random.randint(0, 10, n), 
                bsl=np.random.randint(50, 400, n)))

In [ ]:
df.sort_values(['person', 'time'])

In [ ]:
%sql DROP TABLE IF EXISTS df

Magic shortcut to creating a database table from `pandas` DataFrame.

In [ ]:
%sql -p df

### Over  creates widows

I've given `pandas` equivalents where possible, but sometimes they are rather unnatural.

In [ ]:
%%sql

SELECT person, time, bsl, row_number() 
OVER () as row_num
FROM df;

In [ ]:
df.assign(row_num =  df.person.expanding(1).count().astype('int'))

### Lag and Lead

In [ ]:
%%sql

SELECT person, time, bsl, 
lag(bsl, 1) OVER () as lag1,
lead(bsl, 2) OVER () as lead2
FROM df;

In [ ]:
df_ = df.copy()
df_['lag1'] = df_.bsl.shift(1)
df_['lead2'] = df_.bsl.shift(-2)
df_

### Order by

In [ ]:
%%sql

SELECT person, time, bsl, row_number() 
OVER (ORDER BY person, time)
FROM df;

In [ ]:
df_ = df.copy()
df_ = df_.sort_values(['person', 'time'])
df_.assign(row_num =  df_.person.expanding(1).count().astype('int'))

### Partition by

In [ ]:
%%sql

SELECT person, time, bsl, row_number() 
OVER (PARTITION BY person ORDER BY time) as row_number
FROM df;

In [ ]:
df_ = df.copy()
df_ = df_.sort_values(['person', 'time'])
df_['row_number'] = df_.groupby(['person']).cumcount()+1
df_

In [ ]:
%%sql

SELECT person, time, bsl, group_concat(bsl, ', ')
OVER (PARTITION BY person ORDER BY time) as window
FROM df;

In [ ]:
df_['window'] = (
    df_.groupby('person').
    apply(lambda x: pd.Series([x.bsl.iloc[:(i+1)].values 
                               for i in pd.Series(np.arange(len(x)))]))
).values
df_['window'] = df_.window.apply(lambda x: ','.join(map(str, x)))
df_

### Specifying rows in window

In [ ]:
%%sql

SELECT person, time, bsl, group_concat(bsl, ', ')
OVER (
    PARTITION BY person
    ORDER BY time
    ROWS BETWEEN 1 PRECEDING AND 1 FOLLOWING
     ) as window
FROM df;

In [ ]:
df_['window'] = (
    df_.groupby('person').
    apply(lambda x: pd.Series([x.bsl.iloc[(i-len(x)-1):(i+2)].values 
                               for i in pd.Series(np.arange(len(x)))]))
).values
df_['window'] = df_.window.apply(lambda x: ','.join(map(str, x)))
df_

### Using window functions

In [ ]:
%%sql

SELECT person, time, bsl, 
       row_number() OVER win    AS row_number,
       rank() OVER win          AS rank,
       dense_rank() OVER win    AS dense_rank,
       percent_rank() OVER win  AS percent_rank,
       cume_dist() OVER win     AS cume_dist
FROM df
WINDOW win AS (ORDER BY person);

In [ ]:
df_ = df.copy()
df_ = df_.sort_values(['person'])
df_['row_num'] = df_['person'].expanding(1).count().astype('int')
df_['rank'] = df_['person'].rank(method='min').astype('int')
df_['dense_rank'] = df_['person'].rank(method='dense').astype('int')
df_['percent_rank'] = (df_.person.rank(method='min') - 1) / (df_.person.count()-1)
df_['cume_dist'] = df_['person'].rank(method='max', pct=True)
df_

### The NTILE window function

In [ ]:
scores = pd.DataFrame(dict(scores=np.random.randint(0, 10, 10)))
scores

In [ ]:
%sql -p scores

In [ ]:
%%sql

SELECT scores, NTILE(4)
OVER (
    -- PARTITION BY scores
    ORDER BY scores 
) AS quartile
FROM scores

Note: See [this](https://stackoverflow.com/questions/20726493/python-pandas-qcut-behavior-with-of-observations-not-divisible-by-of-bins) for an explanation of the difference between NTILE and `qcut`

In [ ]:
quartiles = pd.qcut(scores.scores, 4, labels=[1,2,3,4])
scores['quantile'] = quartiles
scores.sort_values('scores')

### Using aggregate functions

The `sqlite3` implementation of LAST_VALUE seems to be buggy.

In [ ]:
%%sql

SELECT person, time, bsl,
       SUM(bsl) OVER win    AS bsl_sum,
       AVG(bsl) OVER win    AS bsl_avg,
       MIN(bsl) OVER win    AS bsl_min,
       MAX(bsl) over win as bsl_max,
       FIRST_VALUE(bsl) OVER win as bsl_start,
       LAST_VALUE(bsl) OVER win as bsl_end
FROM df
WINDOW win AS (PARTITION BY person ORDER BY time);

In [ ]:
g = df.groupby('person')
g.first()

In [ ]:
df_ = df.copy()
df_ = df_.sort_values(['person', 'time'])
df_['bsl_sum'] = df_.groupby(['person'])['bsl'].cumsum()
df_['bsl_avg'] = df_.groupby(['person']).expanding().agg({'bsl': 'mean'}).values
df_['bsl_min'] = df_.groupby(['person'])['bsl'].cummin()
df_['bsl_max'] = df_.groupby(['person'])['bsl'].cummax()
df_['bsl_start'] = df_.groupby(['person'])['bsl'].transform('first')
df_['bsl_end'] = df_.groupby(['person'])['bsl'].transform('last')
df_

### Using rows and range to constrain windows

In [ ]:
%%sql

SELECT person, time, bsl, 
       GROUP_CONCAT(CAST(bsl AS TEXT), ', ') OVER win AS vals,
       SUM(bsl) OVER win AS bsl_sum,
       AVG(bsl) OVER win AS bsl_avg
FROM df
WINDOW win AS (
    PARTITION BY person
    ORDER BY time
    ROWS BETWEEN 1 PRECEDING AND 1 FOLLOWING
)
ORDER BY person, time;

In [ ]:
df_ = df.copy()
df_ = df_.sort_values(['person', 'time'])
df_['bsl_sum'] = (df_.groupby('person').
                  bsl.apply(lambda x: x.rolling(3, min_periods=0, center=True).sum()))
df_['bsl_avg'] = (df_.groupby('person').
                  bsl.apply(lambda x: x.rolling(3, min_periods=0, center=True).mean()))
df_

### Frames using Rows and Range

For Range, all rows with the same ORDER BY value are considered peers.

In [ ]:
%%sql

SELECT person, time, bsl, 
       GROUP_CONCAT(CAST(bsl AS TEXT), ', ') OVER win AS vals,
       SUM(bsl) OVER win AS bsl_sum,
       AVG(bsl) OVER win AS bsl_avg
FROM df
WINDOW win AS (
    ORDER BY person, time
    ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
)
ORDER BY person, time;

In [ ]:
df_ = df.copy()
df_ = df_.sort_values(['person', 'time'])
df_['bsl_sum'] = df_.bsl.expanding().sum().astype('int')
df_['bsl_avg'] = df_.bsl.expanding().mean()
df_

In [ ]:
%%sql

SELECT person, time, bsl, 
       GROUP_CONCAT(CAST(bsl AS TEXT), ', ') OVER win AS vals,
       SUM(bsl) OVER win AS bsl_sum,
       AVG(bsl) OVER win AS bsl_avg
FROM df
WINDOW win AS (
    ORDER BY person
    RANGE BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
)
ORDER BY person, time;

In [ ]:
df_ = df.copy()
df_ = df_.sort_values(['person', 'time'])
df_['bsl_sum'] =  df_['bsl'].cumsum().groupby(df_['person']).transform('last')
df_['bsl_sum'] =  df_['bsl'].expanding().mean().groupby(df_['person']).transform('last')
df_